In [3]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [4]:
import os
if not os.path.isdir('saved_models_mnist_{}'.format('lsgan')):
    os.mkdir('saved_models_mnist_{}'.format('lsgan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('lsgan'), mode='w')
import os
import tensorflow as tf


from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class LSGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(z, valid)
        # (!!!) Optimize w.r.t. MSE loss instead of crossentropy
        self.combined.compile(loss='mse', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        # (!!!) No softmax
        model.add(Dense(1))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test,y_test) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        # Rescale -1 to 1
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, global_step,d_loss[0], 100 * d_loss[1], g_loss))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    gan = LSGAN()
    gan.train(epochs=30, batch_size=64, sample_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

epoch:0 step:85 [D loss: 0.070844, acc.: 92.19%] [G loss: 1.001496]
epoch:0 step:86 [D loss: 0.126322, acc.: 87.50%] [G loss: 0.885132]
epoch:0 step:87 [D loss: 0.122390, acc.: 82.81%] [G loss: 0.903748]
epoch:0 step:88 [D loss: 0.173703, acc.: 77.34%] [G loss: 0.949748]
epoch:0 step:89 [D loss: 0.254989, acc.: 71.09%] [G loss: 1.477472]
epoch:0 step:90 [D loss: 0.338487, acc.: 67.97%] [G loss: 0.995543]
epoch:0 step:91 [D loss: 0.384140, acc.: 66.41%] [G loss: 0.863377]
epoch:0 step:92 [D loss: 0.245592, acc.: 75.00%] [G loss: 0.664815]
epoch:0 step:93 [D loss: 0.147238, acc.: 80.47%] [G loss: 0.971815]
epoch:0 step:94 [D loss: 0.097720, acc.: 89.06%] [G loss: 0.860874]
epoch:0 step:95 [D loss: 0.060567, acc.: 98.44%] [G loss: 1.083434]
epoch:0 step:96 [D loss: 0.059065, acc.: 96.88%] [G loss: 1.048466]
epoch:0 step:97 [D loss: 0.040200, acc.: 96.09%] [G loss: 1.011873]
epoch:0 step:98 [D loss: 0.034540, acc.: 99.22%] [G loss: 0.934219]
epoch:0 step:99 [D loss: 0.049864, acc.: 94.53%]

##############
[ 7.88050253  7.59974359 13.25367035 11.80824     9.02500699 11.98333983
 11.27914699 10.77839851  9.47523762 10.29102348]
##########
epoch:0 step:201 [D loss: 0.060375, acc.: 100.00%] [G loss: 1.137801]
epoch:0 step:202 [D loss: 0.196748, acc.: 64.06%] [G loss: 0.731908]
epoch:0 step:203 [D loss: 0.169371, acc.: 67.19%] [G loss: 1.202554]
epoch:0 step:204 [D loss: 0.272716, acc.: 53.91%] [G loss: 0.683808]
epoch:0 step:205 [D loss: 0.159003, acc.: 68.75%] [G loss: 1.019059]
epoch:0 step:206 [D loss: 0.096903, acc.: 95.31%] [G loss: 0.787873]
epoch:0 step:207 [D loss: 0.039472, acc.: 97.66%] [G loss: 1.045727]
epoch:0 step:208 [D loss: 0.023851, acc.: 100.00%] [G loss: 0.950552]
epoch:0 step:209 [D loss: 0.028771, acc.: 97.66%] [G loss: 0.852569]
epoch:0 step:210 [D loss: 0.034208, acc.: 96.09%] [G loss: 0.930938]
epoch:0 step:211 [D loss: 0.027797, acc.: 100.00%] [G loss: 0.944287]
epoch:0 step:212 [D loss: 0.031121, acc.: 97.66%] [G loss: 0.863395]
epoch:0 step:213 [D 

epoch:0 step:318 [D loss: 0.094507, acc.: 85.16%] [G loss: 0.735082]
epoch:0 step:319 [D loss: 0.083810, acc.: 85.16%] [G loss: 0.890853]
epoch:0 step:320 [D loss: 0.070266, acc.: 90.62%] [G loss: 0.889535]
epoch:0 step:321 [D loss: 0.059037, acc.: 89.84%] [G loss: 0.781754]
epoch:0 step:322 [D loss: 0.079054, acc.: 87.50%] [G loss: 0.840330]
epoch:0 step:323 [D loss: 0.077891, acc.: 91.41%] [G loss: 0.886175]
epoch:0 step:324 [D loss: 0.055181, acc.: 89.84%] [G loss: 0.955115]
epoch:0 step:325 [D loss: 0.096903, acc.: 90.62%] [G loss: 0.720278]
epoch:0 step:326 [D loss: 0.077445, acc.: 88.28%] [G loss: 0.982658]
epoch:0 step:327 [D loss: 0.143362, acc.: 78.12%] [G loss: 0.755494]
epoch:0 step:328 [D loss: 0.068749, acc.: 88.28%] [G loss: 0.851471]
epoch:0 step:329 [D loss: 0.102724, acc.: 85.16%] [G loss: 0.783758]
epoch:0 step:330 [D loss: 0.093742, acc.: 82.03%] [G loss: 0.930415]
epoch:0 step:331 [D loss: 0.115190, acc.: 82.81%] [G loss: 0.827893]
epoch:0 step:332 [D loss: 0.059555

epoch:0 step:436 [D loss: 0.043642, acc.: 100.00%] [G loss: 0.993621]
epoch:0 step:437 [D loss: 0.158502, acc.: 78.12%] [G loss: 0.810969]
epoch:0 step:438 [D loss: 0.036087, acc.: 99.22%] [G loss: 1.087070]
epoch:0 step:439 [D loss: 0.144976, acc.: 79.69%] [G loss: 0.788410]
epoch:0 step:440 [D loss: 0.056598, acc.: 96.09%] [G loss: 0.983879]
epoch:0 step:441 [D loss: 0.157409, acc.: 75.78%] [G loss: 0.839864]
epoch:0 step:442 [D loss: 0.049674, acc.: 98.44%] [G loss: 1.077475]
epoch:0 step:443 [D loss: 0.123513, acc.: 81.25%] [G loss: 0.721986]
epoch:0 step:444 [D loss: 0.056888, acc.: 95.31%] [G loss: 0.958360]
epoch:0 step:445 [D loss: 0.082224, acc.: 96.88%] [G loss: 0.855983]
epoch:0 step:446 [D loss: 0.076326, acc.: 89.06%] [G loss: 0.971661]
epoch:0 step:447 [D loss: 0.050291, acc.: 100.00%] [G loss: 0.933890]
epoch:0 step:448 [D loss: 0.107430, acc.: 82.81%] [G loss: 0.716665]
epoch:0 step:449 [D loss: 0.053996, acc.: 99.22%] [G loss: 1.027480]
epoch:0 step:450 [D loss: 0.0989

...

epoch:29 step:27528 [D loss: 0.237852, acc.: 60.16%] [G loss: 0.449920]
epoch:29 step:27529 [D loss: 0.242243, acc.: 61.72%] [G loss: 0.392747]
epoch:29 step:27530 [D loss: 0.217137, acc.: 64.06%] [G loss: 0.435536]
epoch:29 step:27531 [D loss: 0.208556, acc.: 71.09%] [G loss: 0.434965]
epoch:29 step:27532 [D loss: 0.242326, acc.: 63.28%] [G loss: 0.468798]
epoch:29 step:27533 [D loss: 0.202224, acc.: 70.31%] [G loss: 0.459454]
epoch:29 step:27534 [D loss: 0.212241, acc.: 66.41%] [G loss: 0.430112]
epoch:29 step:27535 [D loss: 0.254714, acc.: 56.25%] [G loss: 0.404308]
epoch:29 step:27536 [D loss: 0.213743, acc.: 63.28%] [G loss: 0.410079]
epoch:29 step:27537 [D loss: 0.224704, acc.: 58.59%] [G loss: 0.407877]
epoch:29 step:27538 [D loss: 0.219805, acc.: 66.41%] [G loss: 0.410524]
epoch:29 step:27539 [D loss: 0.227867, acc.: 58.59%] [G loss: 0.433500]
epoch:29 step:27540 [D loss: 0.226066, acc.: 60.94%] [G loss: 0.415511]
epoch:29 step:27541 [D loss: 0.263825, acc.: 55.47%] [G loss: 0.

epoch:29 step:27643 [D loss: 0.210232, acc.: 67.19%] [G loss: 0.472424]
epoch:29 step:27644 [D loss: 0.170776, acc.: 81.25%] [G loss: 0.448410]
epoch:29 step:27645 [D loss: 0.199372, acc.: 73.44%] [G loss: 0.484862]
epoch:29 step:27646 [D loss: 0.254242, acc.: 53.91%] [G loss: 0.433094]
epoch:29 step:27647 [D loss: 0.212570, acc.: 63.28%] [G loss: 0.437844]
epoch:29 step:27648 [D loss: 0.192773, acc.: 70.31%] [G loss: 0.477293]
epoch:29 step:27649 [D loss: 0.202560, acc.: 71.88%] [G loss: 0.465533]
epoch:29 step:27650 [D loss: 0.261301, acc.: 50.78%] [G loss: 0.441765]
epoch:29 step:27651 [D loss: 0.220393, acc.: 64.06%] [G loss: 0.392441]
epoch:29 step:27652 [D loss: 0.227459, acc.: 64.06%] [G loss: 0.403384]
epoch:29 step:27653 [D loss: 0.218577, acc.: 63.28%] [G loss: 0.402375]
epoch:29 step:27654 [D loss: 0.195364, acc.: 66.41%] [G loss: 0.421063]
epoch:29 step:27655 [D loss: 0.253209, acc.: 53.91%] [G loss: 0.432049]
epoch:29 step:27656 [D loss: 0.217143, acc.: 63.28%] [G loss: 0.

epoch:29 step:27762 [D loss: 0.200282, acc.: 71.09%] [G loss: 0.452099]
epoch:29 step:27763 [D loss: 0.226123, acc.: 61.72%] [G loss: 0.442227]
epoch:29 step:27764 [D loss: 0.247029, acc.: 53.12%] [G loss: 0.405875]
epoch:29 step:27765 [D loss: 0.213278, acc.: 67.19%] [G loss: 0.435649]
epoch:29 step:27766 [D loss: 0.204452, acc.: 65.62%] [G loss: 0.461890]
epoch:29 step:27767 [D loss: 0.244517, acc.: 56.25%] [G loss: 0.426987]
epoch:29 step:27768 [D loss: 0.215568, acc.: 68.75%] [G loss: 0.413683]
epoch:29 step:27769 [D loss: 0.245555, acc.: 55.47%] [G loss: 0.411128]
epoch:29 step:27770 [D loss: 0.238628, acc.: 60.16%] [G loss: 0.470301]
epoch:29 step:27771 [D loss: 0.209529, acc.: 62.50%] [G loss: 0.472275]
epoch:29 step:27772 [D loss: 0.222388, acc.: 60.94%] [G loss: 0.431730]
epoch:29 step:27773 [D loss: 0.258214, acc.: 53.91%] [G loss: 0.405506]
epoch:29 step:27774 [D loss: 0.231946, acc.: 61.72%] [G loss: 0.407675]
epoch:29 step:27775 [D loss: 0.241783, acc.: 57.81%] [G loss: 0.

epoch:29 step:27880 [D loss: 0.213436, acc.: 67.97%] [G loss: 0.457862]
epoch:29 step:27881 [D loss: 0.206147, acc.: 68.75%] [G loss: 0.411217]
epoch:29 step:27882 [D loss: 0.205468, acc.: 67.19%] [G loss: 0.453180]
epoch:29 step:27883 [D loss: 0.236371, acc.: 57.03%] [G loss: 0.425575]
epoch:29 step:27884 [D loss: 0.233211, acc.: 58.59%] [G loss: 0.410277]
epoch:29 step:27885 [D loss: 0.227259, acc.: 58.59%] [G loss: 0.440865]
epoch:29 step:27886 [D loss: 0.232372, acc.: 58.59%] [G loss: 0.420118]
epoch:29 step:27887 [D loss: 0.226290, acc.: 64.84%] [G loss: 0.429865]
epoch:29 step:27888 [D loss: 0.209102, acc.: 65.62%] [G loss: 0.396773]
epoch:29 step:27889 [D loss: 0.232374, acc.: 57.03%] [G loss: 0.415032]
epoch:29 step:27890 [D loss: 0.216475, acc.: 67.19%] [G loss: 0.408778]
epoch:29 step:27891 [D loss: 0.237293, acc.: 57.81%] [G loss: 0.432451]
epoch:29 step:27892 [D loss: 0.193101, acc.: 70.31%] [G loss: 0.452507]
epoch:29 step:27893 [D loss: 0.223751, acc.: 60.16%] [G loss: 0.

epoch:29 step:27994 [D loss: 0.245679, acc.: 58.59%] [G loss: 0.394205]
epoch:29 step:27995 [D loss: 0.221985, acc.: 62.50%] [G loss: 0.437782]
epoch:29 step:27996 [D loss: 0.198274, acc.: 73.44%] [G loss: 0.429000]
epoch:29 step:27997 [D loss: 0.225694, acc.: 62.50%] [G loss: 0.403978]
epoch:29 step:27998 [D loss: 0.219771, acc.: 64.84%] [G loss: 0.398571]
epoch:29 step:27999 [D loss: 0.225406, acc.: 65.62%] [G loss: 0.401325]
epoch:29 step:28000 [D loss: 0.238862, acc.: 56.25%] [G loss: 0.418704]
##############
[2.65194832 1.92995078 6.12647604 4.81225193 3.60127842 5.4784765
 4.50242794 5.13158729 4.46582561 3.95251836]
##########
epoch:29 step:28001 [D loss: 0.230660, acc.: 62.50%] [G loss: 0.477472]
epoch:29 step:28002 [D loss: 0.235174, acc.: 59.38%] [G loss: 0.429572]
epoch:29 step:28003 [D loss: 0.221488, acc.: 58.59%] [G loss: 0.413282]
epoch:29 step:28004 [D loss: 0.240506, acc.: 59.38%] [G loss: 0.383790]
epoch:29 step:28005 [D loss: 0.204363, acc.: 67.97%] [G loss: 0.409042